In [1]:
# main.py script downloads all necessary files from MnDOT site.
%run -i "main.py"

In [2]:
import pandas as pd
import requests
import numpy as np
from io import BytesIO

# Active Stations

In [3]:
filename = 'data/active_stations.pkl'
try:
    active_df = pd.read_pickle(filename)
except:
    # Active station file not found. Download it from MnDOT website.
    url = 'https://www.dot.state.mn.us/traffic/data/reports/Current_CC_StationList.xlsx'
    response = requests.get(url, allow_redirects=True)
    active_df = pd.read_excel(BytesIO(response.content))
    active_df.to_pickle(filename)

active_df.head()

,Continuous Number,Sequence Number,Collection Type,Route,Pos Dir Dir,Pos Lanes,Neg Lanes,Urban/Rural,Functional Class,County Name,Location Text
0,26,5707,WIM,I-35,North,2,2,Rural,Interstate,Steele,"3.5 MI N OF TH30, N OF ELLENDALE"
1,27,9830,WIM,TH 60,East,2,2,Rural,Principal Arterial - Other,Watonwan,"0.7 MI W OF W JCT OF TH4, SW OF ST JAMES"
2,28,22993,"ATR Volume, Speed, Class",MSAS 114,East,1,1,Urban,Minor Arterial,Stearns,W OF 20TH AVE N IN ST CLOUD
3,29,69377,WIM,TH 53,North,2,2,Rural,Principal Arterial - Other,Saint Louis,"3 MI S OF CSAH59 (MELRUDE RD), S OF EVELETH"
4,30,6757,WIM,TH 61,North,2,2,Rural,Principal Arterial - Other,Lake,"SW OF CSAH25, SW OF TWO HARBORS"


# Data Cleaning

We begin with a frame which maintains the format seen in the csv files posted beginning in 2017. We would like to manipulate the data so that each column becomes a time series giving monthly average volumes for each station. 

In [4]:
filename = 'data/full_data.pkl'
df = pd.read_pickle(filename)
df.head()

,station_id,dir_of_travel,lane_of_travel,date,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
0,8,3,0,2002-01-01 00:00:00,3,12,4,4,4,3,...,10,19,12,17,11,9,8,10,16,5
1,8,3,0,2002-01-02 00:00:00,2,0,0,1,0,3,...,39,25,21,22,16,16,13,10,0,6
2,8,3,0,2002-01-03 00:00:00,2,0,2,0,2,4,...,22,34,33,33,61,20,13,20,3,0
3,8,3,0,2002-01-04 00:00:00,4,1,1,0,0,3,...,21,40,47,94,76,48,25,49,25,28
4,8,3,0,2002-01-05 00:00:00,6,7,3,1,0,2,...,19,23,20,10,15,16,16,8,14,8


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169797 entries, 0 to 78946
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   station_id      1169797 non-null  object
 1   dir_of_travel   1169797 non-null  object
 2   lane_of_travel  1169797 non-null  object
 3   date            1169797 non-null  object
 4   1               1169797 non-null  object
 5   2               1169797 non-null  object
 6   3               1169797 non-null  object
 7   4               1169797 non-null  object
 8   5               1169797 non-null  object
 9   6               1169797 non-null  object
 10  7               1169797 non-null  object
 11  8               1169797 non-null  object
 12  9               1169797 non-null  object
 13  10              1169797 non-null  object
 14  11              1169797 non-null  object
 15  12              1169797 non-null  object
 16  13              1169797 non-null  object
 17  14        

## Elimination of Duplicate Records

There is some duplication in the data due to a source file accidentally duplicating another. We eliminate the
duplication.

In [6]:
# Station 53 was duplicated in 2007 (Station 51 information is erroneous).
df_dups = df.duplicated(keep='first')
df[df_dups]

,station_id,dir_of_travel,lane_of_travel,date,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,24
0,53,1,0,2007-01-01 00:00:00,18,14,12,7,6,10,...,19,15,12,13,11,7,8,8,11,11
1,53,1,0,2007-01-02 00:00:00,7,9,8,7,7,13,...,13,11,13,8,10,15,8,5,0,0
2,53,1,0,2007-01-03 00:00:00,1,0,2,0,5,2,...,26,14,13,16,20,12,14,12,10,13
3,53,1,0,2007-01-04 00:00:00,10,9,11,10,7,7,...,14,21,13,15,14,12,13,14,10,11
4,53,1,0,2007-01-05 00:00:00,11,11,13,10,7,10,...,25,19,9,14,3,9,2,4,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,53,5,0,2007-12-27 00:00:00,1,1,0,0,0,0,...,12,17,15,21,8,8,8,9,4,6
726,53,5,0,2007-12-28 00:00:00,0,0,0,0,1,1,...,13,16,16,10,10,7,7,6,2,10
727,53,5,0,2007-12-29 00:00:00,2,1,0,0,1,1,...,11,9,6,6,6,7,7,5,6,3
728,53,5,0,2007-12-30 00:00:00,1,1,0,1,0,0,...,6,11,11,7,7,7,5,9,6,4


In [7]:
df = df[~df_dups]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169067 entries, 0 to 78946
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   station_id      1169067 non-null  object
 1   dir_of_travel   1169067 non-null  object
 2   lane_of_travel  1169067 non-null  object
 3   date            1169067 non-null  object
 4   1               1169067 non-null  object
 5   2               1169067 non-null  object
 6   3               1169067 non-null  object
 7   4               1169067 non-null  object
 8   5               1169067 non-null  object
 9   6               1169067 non-null  object
 10  7               1169067 non-null  object
 11  8               1169067 non-null  object
 12  9               1169067 non-null  object
 13  10              1169067 non-null  object
 14  11              1169067 non-null  object
 15  12              1169067 non-null  object
 16  13              1169067 non-null  object
 17  14        

We need to correct datatypes. The date column should be a datetime, all other columns should be integer type.


In [8]:
df.loc[:, 'date'] = df.loc[:, 'date'].apply(pd.to_datetime)

Later on, we need to know the first and last days covered by the dataset. Rather than hard-coding those, we will pull those out now.

In [9]:
first_date = df.date.min()
end_date = df.date.max()
print(f'First date: {first_date}; last date: {end_date}')

First date: 2002-01-01 00:00:00; last date: 2021-07-31 00:00:00


In [10]:
non_numeric_columns = ['date']
df.loc[:, ~df.columns.isin(non_numeric_columns)] \
        = df.loc[:, ~df.columns.isin(non_numeric_columns)].apply(pd.to_numeric)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1169067 entries, 0 to 78946
Data columns (total 28 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   station_id      1169067 non-null  int64         
 1   dir_of_travel   1169067 non-null  int64         
 2   lane_of_travel  1169067 non-null  int64         
 3   date            1169067 non-null  datetime64[ns]
 4   1               1169067 non-null  int64         
 5   2               1169067 non-null  int64         
 6   3               1169067 non-null  int64         
 7   4               1169067 non-null  int64         
 8   5               1169067 non-null  int64         
 9   6               1169067 non-null  int64         
 10  7               1169067 non-null  int64         
 11  8               1169067 non-null  int64         
 12  9               1169067 non-null  int64         
 13  10              1169067 non-null  int64         
 14  11              1169

## Fold Hour Columns

There are twenty-four columns of traffic volume counts, one for each hour of the day. We would like to
adjust this so that there is one column of counts, with a row for each hour.

In [12]:
df = pd.melt(df, id_vars=['station_id', 'dir_of_travel', 'lane_of_travel', 'date'],
        var_name='hour', value_name='count')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28057608 entries, 0 to 28057607
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   station_id      int64         
 1   dir_of_travel   int64         
 2   lane_of_travel  int64         
 3   date            datetime64[ns]
 4   hour            object        
 5   count           int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 1.3+ GB


In [14]:
df.head()

,station_id,dir_of_travel,lane_of_travel,date,hour,count
0,8,3,0,2002-01-01,1,3
1,8,3,0,2002-01-02,1,2
2,8,3,0,2002-01-03,1,2
3,8,3,0,2002-01-04,1,4
4,8,3,0,2002-01-05,1,6


To simplify later calculations, we break the date into its component parts and include the day of the week as well.

In [15]:
df['month'] = df.date.dt.month
df['day'] = df.date.dt.day
df['year'] = df.date.dt.year
df['day_of_week'] = df.date.dt.dayofweek

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28057608 entries, 0 to 28057607
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   station_id      int64         
 1   dir_of_travel   int64         
 2   lane_of_travel  int64         
 3   date            datetime64[ns]
 4   hour            object        
 5   count           int64         
 6   month           int64         
 7   day             int64         
 8   year            int64         
 9   day_of_week     int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 2.1+ GB


In [17]:
df.head()

,station_id,dir_of_travel,lane_of_travel,date,hour,count,month,day,year,day_of_week
0,8,3,0,2002-01-01,1,3,1,1,2002,1
1,8,3,0,2002-01-02,1,2,1,2,2002,2
2,8,3,0,2002-01-03,1,2,1,3,2002,3
3,8,3,0,2002-01-04,1,4,1,4,2002,4
4,8,3,0,2002-01-05,1,6,1,5,2002,5


We will not use either the lane information or the direction of travel, so we aggregate over those values. We also are maintaining the date in other columns, so we can drop that for the time being as well.

In [18]:
df = df.groupby(['station_id', 'year', 'month', 'day', 'hour', 'day_of_week'], as_index=False).sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13069128 entries, 0 to 13069127
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   station_id      int64 
 1   year            int64 
 2   month           int64 
 3   day             int64 
 4   hour            object
 5   day_of_week     int64 
 6   dir_of_travel   int64 
 7   lane_of_travel  int64 
 8   count           int64 
dtypes: int64(8), object(1)
memory usage: 897.4+ MB


In [19]:
df.head()

,station_id,year,month,day,hour,day_of_week,dir_of_travel,lane_of_travel,count
0,8,2002,1,1,1,1,10,0,9
1,8,2002,1,1,10,1,10,0,14
2,8,2002,1,1,11,1,10,0,14
3,8,2002,1,1,12,1,10,0,13
4,8,2002,1,1,13,1,10,0,26


In [20]:
df = df.drop(['dir_of_travel', 'lane_of_travel'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13069128 entries, 0 to 13069127
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   station_id   int64 
 1   year         int64 
 2   month        int64 
 3   day          int64 
 4   hour         object
 5   day_of_week  int64 
 6   count        int64 
dtypes: int64(6), object(1)
memory usage: 698.0+ MB


In [21]:
df.head()

,station_id,year,month,day,hour,day_of_week,count
0,8,2002,1,1,1,1,9
1,8,2002,1,1,10,1,14
2,8,2002,1,1,11,1,14
3,8,2002,1,1,12,1,13
4,8,2002,1,1,13,1,26


In [22]:
df['count'].describe()

count    1.306913e+07
mean     1.175178e+03
std      1.994335e+03
min      0.000000e+00
25%      6.300000e+01
50%      3.190000e+02
75%      1.340000e+03
max      1.690100e+04
Name: count, dtype: float64

## Remove Inactive Stations

We drop any station that isn't currently active.

In [23]:
active_stations = active_df['Continuous Number'].values

In [24]:
is_active = df.station_id.isin(active_stations)

In [25]:
df = df[is_active]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11349336 entries, 135840 to 12957119
Data columns (total 7 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   station_id   int64 
 1   year         int64 
 2   month        int64 
 3   day          int64 
 4   hour         object
 5   day_of_week  int64 
 6   count        int64 
dtypes: int64(6), object(1)
memory usage: 692.7+ MB


In [27]:
df['count'].describe()

count    1.134934e+07
mean     1.267724e+03
std      2.075760e+03
min      0.000000e+00
25%      7.800000e+01
50%      3.750000e+02
75%      1.470000e+03
max      1.690100e+04
Name: count, dtype: float64

## Pivot Stations to Columns

Pivot so that all columns are essentially time series, with one column per station. We keep the hours separate to assist with later grouping.

In [28]:
df = df.pivot(index=['year', 'month', 'day', 'hour', 'day_of_week'], columns='station_id', values='count').reset_index()

In [29]:
# station_id is currently naming all columns. We'll get rid of that.
df.columns.name = ''

In [30]:
df.head()

,year,month,day,hour,day_of_week,26,27,28,29,30,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
0,2002,1,1,1,1,NaN,NaN,166.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,1,1,10,1,NaN,NaN,191.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,1,1,11,1,NaN,NaN,248.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,1,1,12,1,NaN,NaN,336.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,1,1,13,1,NaN,NaN,397.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.tail()

,year,month,day,hour,day_of_week,26,27,28,29,30,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
171547,2021,7,31,5,5,NaN,NaN,29.0,NaN,48.0,...,17.0,4.0,NaN,85.0,102.0,NaN,NaN,11.0,34.0,125.0
171548,2021,7,31,6,5,NaN,NaN,48.0,NaN,62.0,...,31.0,4.0,NaN,135.0,159.0,NaN,NaN,27.0,30.0,108.0
171549,2021,7,31,7,5,NaN,NaN,87.0,NaN,167.0,...,44.0,16.0,NaN,232.0,232.0,NaN,NaN,47.0,90.0,178.0
171550,2021,7,31,8,5,NaN,NaN,101.0,NaN,272.0,...,83.0,20.0,NaN,415.0,373.0,NaN,NaN,66.0,143.0,255.0
171551,2021,7,31,9,5,NaN,NaN,228.0,NaN,433.0,...,109.0,52.0,NaN,567.0,607.0,NaN,NaN,136.0,280.0,480.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171552 entries, 0 to 171551
Columns: 108 entries, year to 6224
dtypes: float64(103), int64(4), object(1)
memory usage: 141.4+ MB


# Calculate MADT

While hourly counts can be used for short-term predictions, we will be looking at longer term changes in this project. For this reason, we will use monthly values. The standard measure used in traffic management is the annual average daily traffic (AADT). The [Federal Highway Administration](https://www.fhwa.dot.gov/policyinformation/tmguide/tmg_fhwa_pl_17_003.pdf) recommends first calculating the monthly average daily traffic (MADT) and using these to produce the AADT. The MADT isn't quite the average of the daily totals over the month because of weighting factors meant to account for missing data.

The MADT is given by the formula
$$
MADT_m = \frac{\sum_{j=1}^7 w_{jm} \sum_{h=1}^{24}\left[\frac{1}{n_{hjm}}\sum_{i=1}^{n_{hjm}}VOL_{ihjm}\right]}{\sum_{j=1}^7 w_{jm}}
$$
where $m$ is the month (represented as an integer between 1 and 12), $j$ is the day of the week (represented as an integer between 1 and 7), $h$ is the hour of the day (as an integer from 1 to 24), $w_{jm}$ is the number of times the $j$th day of the week occurs in month $m$, $n_{hjm}$ is the number of available data points for the $h$th hour of the $j$th day of the week in month $m$ (between 1 and 5), $VOL_{ihjm}$ is the $i$th data point for the $h$th hour of the $j$th day of the week in month $m$, and $MADT_m$ is the monthly average daily traffic for month $m$. The particular choice of which day of the week and hour of the day are represented by 1 doesn't affect the result. 

In [33]:
# In each month, perform the following. 1. For each day of the week and each hour of the day, find the mean of the available 
# data points
df = df.groupby(['year', 'month', 'hour', 'day_of_week'], as_index=False).mean().reset_index(drop=True)
df = df.drop(['day'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39480 entries, 0 to 39479
Columns: 107 entries, year to 6224
dtypes: float64(103), int64(3), object(1)
memory usage: 32.2+ MB


In [34]:
df.head()

,year,month,hour,day_of_week,26,27,28,29,30,31,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
0,2002,1,1,0,NaN,NaN,50.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,1,1,1,NaN,NaN,78.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,1,1,2,NaN,NaN,54.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,1,1,3,NaN,NaN,74.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,1,1,4,NaN,NaN,90.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Continue MADT process. 2. Sum over every hour of the day. The min_count=1 argument keeps NaN for any duration where there
# are no valid counts.
df = df.groupby(['year', 'month', 'day_of_week'], as_index=False).sum(min_count=1)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1645 entries, 0 to 1644
Columns: 106 entries, year to 6224
dtypes: float64(103), int64(3)
memory usage: 1.3 MB


In [37]:
df.head()

,year,month,day_of_week,26,27,28,29,30,31,32,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
0,2002,1,0,NaN,NaN,10071.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,1,1,NaN,NaN,9428.60,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,1,2,NaN,NaN,10530.40,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,1,3,NaN,NaN,10704.40,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,1,4,NaN,NaN,11675.25,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df.tail()

,year,month,day_of_week,26,27,28,29,30,31,32,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
1640,2021,7,2,NaN,7587.000000,8407.75,NaN,10571.00,5208.0,31696.666667,...,4593.0,1571.50,12752.25,24121.25,18574.50,NaN,NaN,4246.50,7774.25,14574.25
1641,2021,7,3,NaN,8462.500000,8428.60,NaN,11633.80,6333.0,34499.500000,...,4877.0,1992.40,13984.20,25412.75,20495.20,NaN,NaN,4778.25,9003.60,16286.50
1642,2021,7,4,NaN,10105.666667,8534.20,NaN,12701.00,7237.0,39314.250000,...,4951.0,2886.60,15458.75,25874.25,23473.00,NaN,NaN,5671.20,10884.20,19199.60
1643,2021,7,5,NaN,8012.333333,6066.60,NaN,11335.00,4672.0,33015.400000,...,3561.0,1857.60,12267.00,19705.00,18506.00,NaN,NaN,5251.75,8274.20,15010.40
1644,2021,7,6,NaN,8247.000000,5140.25,NaN,10944.25,3812.0,30810.000000,...,3125.0,2202.25,11872.50,17560.25,18833.25,NaN,NaN,5370.50,8413.00,15949.00


In [39]:
# Now, we need a weighted average over days of the week. We'll produce a second dataframe where we count the number of 
# days of the week for each month in the study period.
dates = pd.date_range(first_date, end_date)
days_of_week = pd.DataFrame({'year': dates.year, 'month': dates.month, 'day_of_week': dates.dayofweek, 'dow_count': 1})
days_of_week = days_of_week.groupby(['year', 'month', 'day_of_week']).count()

In [40]:
days_of_week.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1645 entries, (2002, 1, 0) to (2021, 7, 6)
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   dow_count  1645 non-null   int64
dtypes: int64(1)
memory usage: 18.1 KB


In [41]:
df = df.join(days_of_week, on=['year', 'month', 'day_of_week'])

In [42]:
# The weighted average itself takes a little work to get. We'll move the weights into the data points, group, then divide
# by the sum of the weights.
df.loc[:,df.columns[3:-1]] = df.loc[:,df.columns[3:-1]].transform(lambda x: x*df.loc[x.index, "dow_count"])

In [43]:
df.tail()

,year,month,day_of_week,26,27,28,29,30,31,32,...,3371,3467,3790,3801,4820,4910,5984,6067,6224,dow_count
1640,2021,7,2,NaN,30348.000000,33631.0,NaN,42284.0,20832.0,126786.666667,...,6286.0,51009.00,96485.00,74298.0,NaN,NaN,16986.00,31097.0,58297.0,4
1641,2021,7,3,NaN,42312.500000,42143.0,NaN,58169.0,31665.0,172497.500000,...,9962.0,69921.00,127063.75,102476.0,NaN,NaN,23891.25,45018.0,81432.5,5
1642,2021,7,4,NaN,50528.333333,42671.0,NaN,63505.0,36185.0,196571.250000,...,14433.0,77293.75,129371.25,117365.0,NaN,NaN,28356.00,54421.0,95998.0,5
1643,2021,7,5,NaN,40061.666667,30333.0,NaN,56675.0,23360.0,165077.000000,...,9288.0,61335.00,98525.00,92530.0,NaN,NaN,26258.75,41371.0,75052.0,5
1644,2021,7,6,NaN,32988.000000,20561.0,NaN,43777.0,15248.0,123240.000000,...,8809.0,47490.00,70241.00,75333.0,NaN,NaN,21482.00,33652.0,63796.0,4


In [44]:
df = df.groupby(['year', 'month'], as_index=False).sum(min_count=1)
df.tail()

,year,month,day_of_week,26,27,28,29,30,31,32,...,3371,3467,3790,3801,4820,4910,5984,6067,6224,dow_count
230,2021,3,21,593639.250000,NaN,209198.666667,NaN,222667.5,143407.000000,8.538553e+05,...,32698.333333,301677.000000,660674.0,454736.000000,348226.666667,184292.000000,97896.666667,202219.583333,348941.000000,31
231,2021,4,21,623920.000000,193767.000000,209485.000000,NaN,221090.0,142713.000000,9.072242e+05,...,37641.333333,319808.333333,656406.0,487543.750000,369533.000000,185941.666667,131044.000000,206918.166667,365602.750000,30
232,2021,5,21,694553.000000,227017.750000,226678.000000,NaN,278877.0,158551.333333,9.798203e+05,...,55226.000000,348916.333333,685957.0,536508.666667,424256.000000,215445.000000,149193.000000,238064.750000,423502.666667,31
233,2021,6,21,744758.666667,230493.916667,230293.000000,NaN,321349.0,162088.333333,1.007206e+06,...,55644.000000,388756.000000,701268.0,579805.000000,452443.333333,232838.833333,147930.833333,253077.666667,463829.833333,30
234,2021,7,21,NaN,257721.166667,233637.000000,NaN,352269.0,173390.000000,1.034379e+06,...,65237.000000,408611.750000,707393.0,609424.000000,NaN,NaN,152347.000000,270316.000000,491918.833333,31


In [45]:
df.loc[:,df.columns[3:-1]] = df.loc[:,df.columns[3:-1]].transform(lambda x: x/df.loc[x.index, "dow_count"])

In [46]:
df.tail()

,year,month,day_of_week,26,27,28,29,30,31,32,...,3371,3467,3790,3801,4820,4910,5984,6067,6224,dow_count
230,2021,3,21,19149.653226,NaN,6748.344086,NaN,7182.822581,4626.032258,27543.720430,...,1054.784946,9731.516129,21312.064516,14668.903226,11233.118280,5944.903226,3157.956989,6523.212366,11256.161290,31
231,2021,4,21,20797.333333,6458.900000,6982.833333,NaN,7369.666667,4757.100000,30240.808333,...,1254.711111,10660.277778,21880.200000,16251.458333,12317.766667,6198.055556,4368.133333,6897.272222,12186.758333,30
232,2021,5,21,22404.935484,7323.153226,7312.193548,NaN,8996.032258,5114.559140,31607.107527,...,1781.483871,11255.365591,22127.645161,17306.731183,13685.677419,6949.838710,4812.677419,7679.508065,13661.376344,31
233,2021,6,21,24825.288889,7683.130556,7676.433333,NaN,10711.633333,5402.944444,33573.533333,...,1854.800000,12958.533333,23375.600000,19326.833333,15081.444444,7761.294444,4931.027778,8435.922222,15460.994444,30
234,2021,7,21,NaN,8313.586022,7536.677419,NaN,11363.516129,5593.225806,33367.067204,...,2104.419355,13181.024194,22819.129032,19658.838710,NaN,NaN,4914.419355,8719.870968,15868.349462,31


In [47]:
df = df.drop(['day_of_week', 'dow_count'], axis=1)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Columns: 105 entries, year to 6224
dtypes: float64(103), int64(2)
memory usage: 192.9 KB


In [49]:
df.head()

,year,month,26,27,28,29,30,31,32,33,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
0,2002,1,NaN,NaN,9620.677419,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002,2,NaN,NaN,9800.571429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002,3,NaN,NaN,9521.419355,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002,4,NaN,NaN,9661.100000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,5,NaN,NaN,8678.483871,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.tail()

,year,month,26,27,28,29,30,31,32,33,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
230,2021,3,19149.653226,NaN,6748.344086,NaN,7182.822581,4626.032258,27543.720430,4287.491935,...,NaN,1054.784946,9731.516129,21312.064516,14668.903226,11233.118280,5944.903226,3157.956989,6523.212366,11256.161290
231,2021,4,20797.333333,6458.900000,6982.833333,NaN,7369.666667,4757.100000,30240.808333,NaN,...,4175.566667,1254.711111,10660.277778,21880.200000,16251.458333,12317.766667,6198.055556,4368.133333,6897.272222,12186.758333
232,2021,5,22404.935484,7323.153226,7312.193548,NaN,8996.032258,5114.559140,31607.107527,4531.096774,...,4313.129032,1781.483871,11255.365591,22127.645161,17306.731183,13685.677419,6949.838710,4812.677419,7679.508065,13661.376344
233,2021,6,24825.288889,7683.130556,7676.433333,NaN,10711.633333,5402.944444,33573.533333,4742.800000,...,4445.683333,1854.800000,12958.533333,23375.600000,19326.833333,15081.444444,7761.294444,4931.027778,8435.922222,15460.994444
234,2021,7,NaN,8313.586022,7536.677419,NaN,11363.516129,5593.225806,33367.067204,4442.451613,...,4323.096774,2104.419355,13181.024194,22819.129032,19658.838710,NaN,NaN,4914.419355,8719.870968,15868.349462


Now that the entries in the dataframe are MADT values, we can change the index to be a datetime index so that each remaining column is a time series.

In [51]:
date_df = pd.DataFrame({'val': 1}, index=dates)

In [52]:
date_df = date_df.resample('1M').count()

In [53]:
df.index = date_df.index

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 235 entries, 2002-01-31 to 2021-07-31
Freq: M
Columns: 105 entries, year to 6224
dtypes: float64(103), int64(2)
memory usage: 194.6 KB


In [55]:
df.tail()

,year,month,26,27,28,29,30,31,32,33,...,1940,3371,3467,3790,3801,4820,4910,5984,6067,6224
2021-03-31,2021,3,19149.653226,NaN,6748.344086,NaN,7182.822581,4626.032258,27543.720430,4287.491935,...,NaN,1054.784946,9731.516129,21312.064516,14668.903226,11233.118280,5944.903226,3157.956989,6523.212366,11256.161290
2021-04-30,2021,4,20797.333333,6458.900000,6982.833333,NaN,7369.666667,4757.100000,30240.808333,NaN,...,4175.566667,1254.711111,10660.277778,21880.200000,16251.458333,12317.766667,6198.055556,4368.133333,6897.272222,12186.758333
2021-05-31,2021,5,22404.935484,7323.153226,7312.193548,NaN,8996.032258,5114.559140,31607.107527,4531.096774,...,4313.129032,1781.483871,11255.365591,22127.645161,17306.731183,13685.677419,6949.838710,4812.677419,7679.508065,13661.376344
2021-06-30,2021,6,24825.288889,7683.130556,7676.433333,NaN,10711.633333,5402.944444,33573.533333,4742.800000,...,4445.683333,1854.800000,12958.533333,23375.600000,19326.833333,15081.444444,7761.294444,4931.027778,8435.922222,15460.994444
2021-07-31,2021,7,NaN,8313.586022,7536.677419,NaN,11363.516129,5593.225806,33367.067204,4442.451613,...,4323.096774,2104.419355,13181.024194,22819.129032,19658.838710,NaN,NaN,4914.419355,8719.870968,15868.349462


In [56]:
df = df.drop(['year', 'month'], axis=1)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 235 entries, 2002-01-31 to 2021-07-31
Freq: M
Columns: 103 entries, 26 to 6224
dtypes: float64(103)
memory usage: 190.9 KB


# Missing Values

A number of stations have very little data. For example, over a dozen stations have only been collecting from 2019 or later. These stations have almost no data that can be used for training. Consequently, they will be dropped. In particular, any station with 80% or more missing values will be dropped.

In [58]:
row_limit = pd.options.display.max_rows
pd.options.display.max_rows = None
print(df.isna().mean().sort_values(ascending=False))
pd.options.display.max_rows = row_limit

37      0.974468
6067    0.970213
3371    0.970213
47      0.957447
1335    0.931915
1604    0.927660
3467    0.919149
3790    0.914894
3801    0.897872
48      0.880851
4910    0.880851
4820    0.876596
949     0.876596
1940    0.872340
49      0.872340
6224    0.872340
5984    0.872340
495     0.868085
30      0.846809
27      0.795745
46      0.791489
44      0.727660
45      0.714894
32      0.663830
40      0.642553
43      0.617021
42      0.612766
41      0.612766
233     0.587234
31      0.565957
232     0.544681
230     0.544681
231     0.536170
229     0.536170
228     0.523404
38      0.514894
29      0.514894
39      0.514894
34      0.468085
33      0.463830
381     0.425532
35      0.417021
26      0.417021
223     0.323404
342     0.234043
103     0.200000
400     0.200000
354     0.191489
402     0.161702
200     0.161702
407     0.148936
321     0.131915
309     0.131915
315     0.131915
326     0.131915
329     0.131915
335     0.127660
208     0.123404
110     0.1106

In [59]:
row_limit = pd.options.display.max_rows
pd.options.display.max_rows = None
print(df.apply(lambda x: x.first_valid_index()).sort_values(ascending=False))
pd.options.display.max_rows = row_limit

3371   2021-01-31
6067   2021-01-31
3467   2020-01-31
6224   2019-01-31
49     2019-01-31
48     2019-01-31
47     2019-01-31
495    2019-01-31
949    2019-01-31
1335   2019-01-31
1604   2019-01-31
1940   2019-01-31
37     2019-01-31
3790   2019-01-31
3801   2019-01-31
4820   2019-01-31
4910   2019-01-31
5984   2019-01-31
46     2015-02-28
27     2015-01-31
44     2014-01-31
45     2014-01-31
32     2013-01-31
233    2012-12-31
232    2012-09-30
230    2012-09-30
231    2012-07-31
229    2012-07-31
228    2012-04-30
43     2012-01-31
42     2012-01-31
41     2011-01-31
38     2010-01-31
39     2010-01-31
40     2010-01-31
31     2009-08-31
33     2009-01-31
34     2009-01-31
26     2007-01-31
35     2007-01-31
30     2007-01-31
29     2007-01-31
342    2004-01-31
390    2002-01-31
353    2002-01-31
354    2002-01-31
365    2002-01-31
381    2002-01-31
382    2002-01-31
384    2002-01-31
386    2002-01-31
388    2002-01-31
352    2002-01-31
389    2002-01-31
410    2002-01-31
400    200

In [60]:
drop_threshold = 0.80
dropped_filter = df.apply(lambda x: x.isna().mean() >= drop_threshold)
dropped_stations = df.columns[dropped_filter]
df = df.drop(dropped_stations, axis=1)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 235 entries, 2002-01-31 to 2021-07-31
Freq: M
Data columns (total 84 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   26      137 non-null    float64
 1   27      48 non-null     float64
 2   28      211 non-null    float64
 3   29      114 non-null    float64
 4   31      102 non-null    float64
 5   32      79 non-null     float64
 6   33      126 non-null    float64
 7   34      125 non-null    float64
 8   35      137 non-null    float64
 9   38      114 non-null    float64
 10  39      114 non-null    float64
 11  40      84 non-null     float64
 12  41      91 non-null     float64
 13  42      91 non-null     float64
 14  43      90 non-null     float64
 15  44      64 non-null     float64
 16  45      67 non-null     float64
 17  46      49 non-null     float64
 18  51      219 non-null    float64
 19  103     188 non-null    float64
 20  110     209 non-null    float64
 21  164     234 

In [62]:
df.tail()

,26,27,28,29,31,32,33,34,35,38,...,400,402,405,407,410,420,425,458,460,464
2021-03-31,19149.653226,NaN,6748.344086,NaN,4626.032258,27543.720430,4287.491935,2628.473118,NaN,29021.763441,...,NaN,NaN,82354.290323,NaN,3066.204301,27665.516129,17430.860215,7764.698925,3427.139785,17378.075269
2021-04-30,20797.333333,6458.900000,6982.833333,NaN,4757.100000,30240.808333,NaN,3284.227778,NaN,28475.633333,...,NaN,NaN,86072.500000,NaN,3371.133333,28831.033333,19060.233333,8673.600000,2599.633333,18173.494444
2021-05-31,22404.935484,7323.153226,7312.193548,NaN,5114.559140,31607.107527,4531.096774,NaN,NaN,29782.387097,...,NaN,NaN,90431.387097,NaN,3820.129032,29836.967742,19680.080645,9374.225806,3625.032258,19249.473118
2021-06-30,24825.288889,7683.130556,7676.433333,NaN,5402.944444,33573.533333,4742.800000,NaN,7151.655556,32718.200000,...,NaN,NaN,97449.538889,NaN,4122.466667,30650.333333,22119.066667,9219.200000,NaN,20370.433333
2021-07-31,NaN,8313.586022,7536.677419,NaN,5593.225806,33367.067204,4442.451613,611.612903,7406.524194,32919.758065,...,NaN,NaN,96319.387097,NaN,3888.354839,29888.612903,21805.000000,8637.354839,NaN,20055.502688


In [63]:
row_limit = pd.options.display.max_rows
pd.options.display.max_rows = None
print(df.apply(lambda x: x.last_valid_index()).sort_values())
pd.options.display.max_rows = row_limit

381   2015-12-31
40    2016-12-31
400   2017-08-31
402   2018-10-31
354   2018-10-31
309   2018-12-31
315   2018-12-31
321   2018-12-31
326   2018-12-31
329   2018-12-31
223   2018-12-31
342   2018-12-31
407   2019-06-30
341   2019-06-30
208   2020-03-31
175   2020-07-31
352   2020-09-30
29    2020-12-31
219   2020-12-31
41    2021-02-28
200   2021-04-30
460   2021-05-31
211   2021-05-31
303   2021-06-30
26    2021-06-30
191   2021-07-31
42    2021-07-31
39    2021-07-31
335   2021-07-31
336   2021-07-31
38    2021-07-31
35    2021-07-31
351   2021-07-31
34    2021-07-31
353   2021-07-31
33    2021-07-31
365   2021-07-31
32    2021-07-31
43    2021-07-31
382   2021-07-31
386   2021-07-31
388   2021-07-31
389   2021-07-31
390   2021-07-31
31    2021-07-31
28    2021-07-31
405   2021-07-31
27    2021-07-31
410   2021-07-31
420   2021-07-31
425   2021-07-31
458   2021-07-31
384   2021-07-31
187   2021-07-31
44    2021-07-31
305   2021-07-31
198   2021-07-31
199   2021-07-31
179   2021-07-

Additionally, we will drop any station with no updates in the year before the last entries. We take one year to be 365 days for this purpose.

In [64]:
drop_date = end_date - pd.Timedelta(days=365)
dropped_filter = df.apply(lambda x: x.last_valid_index() <= drop_date)
dropped_stations = df.columns[dropped_filter]
df = df.drop(dropped_stations, axis=1)

In [65]:
df.tail()

,26,27,28,29,31,32,33,34,35,38,...,388,389,390,405,410,420,425,458,460,464
2021-03-31,19149.653226,NaN,6748.344086,NaN,4626.032258,27543.720430,4287.491935,2628.473118,NaN,29021.763441,...,17716.903226,34722.021505,9215.086022,82354.290323,3066.204301,27665.516129,17430.860215,7764.698925,3427.139785,17378.075269
2021-04-30,20797.333333,6458.900000,6982.833333,NaN,4757.100000,30240.808333,NaN,3284.227778,NaN,28475.633333,...,18921.333333,35668.733333,10343.600000,86072.500000,3371.133333,28831.033333,19060.233333,8673.600000,2599.633333,18173.494444
2021-05-31,22404.935484,7323.153226,7312.193548,NaN,5114.559140,31607.107527,4531.096774,NaN,NaN,29782.387097,...,20783.354839,36313.548387,11512.838710,90431.387097,3820.129032,29836.967742,19680.080645,9374.225806,3625.032258,19249.473118
2021-06-30,24825.288889,7683.130556,7676.433333,NaN,5402.944444,33573.533333,4742.800000,NaN,7151.655556,32718.200000,...,22065.666667,39609.900000,11792.866667,97449.538889,4122.466667,30650.333333,22119.066667,9219.200000,NaN,20370.433333
2021-07-31,NaN,8313.586022,7536.677419,NaN,5593.225806,33367.067204,4442.451613,611.612903,7406.524194,32919.758065,...,21940.451613,39853.419355,11044.838710,96319.387097,3888.354839,29888.612903,21805.000000,8637.354839,NaN,20055.502688


In [66]:
filename = 'data/preprocessed_counts.pkl'
df.to_pickle(filename)